# Build a simulation using the Python API

Currently, this notbook manipulates the simulation by directly placing objects inside of the attributes of the network and domain. It should be refactored when proper methods exist for adding these objects.


Import the Simulation class

In [1]:
from primaite.simulator.sim_container import Simulation


Create an empty simulation. By default this has a network with no nodes or links, and a domain controller with no accounts.

Let's use the simulation's `describe_state()` method to verify that it is empty.

In [2]:
my_sim = Simulation()
net = my_sim.network
my_sim.describe_state()

{'uuid': '95929b6a-1ce4-4c94-966c-6d3246d7caf9',
 'network': {'uuid': '4b41398e-d768-47c5-80cf-4278cfc35a24',
  'nodes': {},
  'links': {}},
 'domain': {'uuid': '15920e15-6cd1-4a93-b6af-acbcc6f6468e', 'accounts': {}}}

## Add nodes

In [3]:
from primaite.simulator.network.hardware.base import Node


In [4]:
my_pc = Node(hostname="primaite_pc",)
net.add_node(my_pc)
my_server = Node(hostname="google_server")
net.add_node(my_server)


## Connect the nodes

In [5]:
from primaite.simulator.network.hardware.base import NIC, Link, Switch


In [6]:
my_swtich = Switch(hostname="switch1", num_ports=12)
net.add_node(my_swtich)

pc_nic = NIC(ip_address="130.1.1.1", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_pc.connect_nic(pc_nic)


server_nic = NIC(ip_address="130.1.1.2", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_server.connect_nic(server_nic)


pc_to_switch = Link(endpoint_a=pc_nic, endpoint_b=my_swtich.switch_ports[1])
server_to_swtich = Link(endpoint_a=server_nic, endpoint_b=my_swtich.switch_ports[2])

my_sim.network.links[pc_to_switch.uuid] = pc_to_switch
my_sim.network.links[server_to_swtich.uuid] = server_to_swtich

2023-08-23 15:44:02,059: NIC 1b:8f:94:4f:46:99/130.1.1.1 connected to Link 1b:8f:94:4f:46:99/130.1.1.1<-->ad:3c:77:44:98:27
2023-08-23 15:44:02,062: SwitchPort ad:3c:77:44:98:27 connected to Link 1b:8f:94:4f:46:99/130.1.1.1<-->ad:3c:77:44:98:27
2023-08-23 15:44:02,064: NIC 50:f4:6b:9b:a8:74/130.1.1.2 connected to Link 50:f4:6b:9b:a8:74/130.1.1.2<-->fd:b1:68:f9:8f:eb
2023-08-23 15:44:02,065: SwitchPort fd:b1:68:f9:8f:eb connected to Link 50:f4:6b:9b:a8:74/130.1.1.2<-->fd:b1:68:f9:8f:eb


## Add files and folders to nodes


In [7]:
from primaite.simulator.file_system.file_system_file_type import FileSystemFileType
from primaite.simulator.file_system.file_system_file import FileSystemFile

In [8]:
my_pc_downloads_folder = my_pc.file_system.create_folder("downloads")
my_pc_downloads_folder.add_file(FileSystemFile(name="firefox_installer.zip",file_type=FileSystemFileType.ZIP))

In [9]:
my_server_folder = my_server.file_system.create_folder("static")
my_server.file_system.create_file("favicon.ico", file_type=FileSystemFileType.PNG)

FileSystemFile(uuid='f45bffd7-4aa1-4f6f-81ba-85e746abd28b', name='favicon.ico', size=40.0, file_type=<FileSystemFileType.PNG: '11'>, action_manager=None)

## Add applications to nodes

In [10]:
from primaite.simulator.system.applications.application import Application, ApplicationOperatingState
from primaite.simulator.system.software import SoftwareHealthState, SoftwareCriticality
from primaite.simulator.network.transmission.transport_layer import Port

# no applications exist yet so we will create our own.
class MSPaint(Application):
    def describe_state(self):
        return super().describe_state()

In [11]:
mspaint = MSPaint(name = "mspaint", health_state_actual=SoftwareHealthState.GOOD, health_state_visible=SoftwareHealthState.GOOD, criticality=SoftwareCriticality.MEDIUM, ports={Port.HTTP}, operating_state=ApplicationOperatingState.RUNNING,execution_control_status='manual')

In [12]:
my_pc.applications[mspaint.uuid] = mspaint

## Create a domain account

In [13]:
from primaite.simulator.domain.account import Account, AccountType


In [14]:
acct = Account(username="admin", password="admin12", account_type=AccountType.USER)
my_sim.domain.accounts[acct.uuid] = acct

Verify that the state dictionary contains no non-serialisable objects.

In [15]:
my_sim.describe_state()

{'uuid': '95929b6a-1ce4-4c94-966c-6d3246d7caf9',
 'network': {'uuid': '4b41398e-d768-47c5-80cf-4278cfc35a24',
  'nodes': {'1599c08e-a101-41a7-a86a-4176660c4270': {'uuid': '1599c08e-a101-41a7-a86a-4176660c4270',
    'hostname': 'primaite_pc',
    'operating_state': 0,
    'NICs': {'ab09d298-ac44-40ef-b950-b4ca6268d482': {'uuid': 'ab09d298-ac44-40ef-b950-b4ca6268d482',
      'ip_adress': '130.1.1.1',
      'subnet_mask': '255.255.255.0',
      'gateway': '130.1.1.255',
      'mac_address': '1b:8f:94:4f:46:99',
      'speed': 100,
      'mtu': 1500,
      'wake_on_lan': False,
      'dns_servers': [],
      'enabled': False}},
    'file_system': {'uuid': '92120387-14cb-426c-98f2-64d64a85f560',
     'folders': {'6a11bd03-bc59-4da9-8474-639fcb72b9be': {'uuid': '6a11bd03-bc59-4da9-8474-639fcb72b9be',
       'name': 'downloads',
       'size': 1000.0,
       'files': {'194b2029-4723-4cff-b6d7-e647e4fb687d': {'uuid': '194b2029-4723-4cff-b6d7-e647e4fb687d',
         'name': 'firefox_installer.z

In [16]:
import json
json.dumps(my_sim.describe_state())

'{"uuid": "95929b6a-1ce4-4c94-966c-6d3246d7caf9", "network": {"uuid": "4b41398e-d768-47c5-80cf-4278cfc35a24", "nodes": {"1599c08e-a101-41a7-a86a-4176660c4270": {"uuid": "1599c08e-a101-41a7-a86a-4176660c4270", "hostname": "primaite_pc", "operating_state": 0, "NICs": {"ab09d298-ac44-40ef-b950-b4ca6268d482": {"uuid": "ab09d298-ac44-40ef-b950-b4ca6268d482", "ip_adress": "130.1.1.1", "subnet_mask": "255.255.255.0", "gateway": "130.1.1.255", "mac_address": "1b:8f:94:4f:46:99", "speed": 100, "mtu": 1500, "wake_on_lan": false, "dns_servers": [], "enabled": false}}, "file_system": {"uuid": "92120387-14cb-426c-98f2-64d64a85f560", "folders": {"6a11bd03-bc59-4da9-8474-639fcb72b9be": {"uuid": "6a11bd03-bc59-4da9-8474-639fcb72b9be", "name": "downloads", "size": 1000.0, "files": {"194b2029-4723-4cff-b6d7-e647e4fb687d": {"uuid": "194b2029-4723-4cff-b6d7-e647e4fb687d", "name": "firefox_installer.zip", "size": 1000.0, "file_type": "ZIP"}}, "is_quarantined": false}}}, "applications": {"ae49273b-f581-44e7